In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Von_Hippel_Lindau"
cohort = "GSE33093"

# Input paths
in_trait_dir = "../../input/GEO/Von_Hippel_Lindau"
in_cohort_dir = "../../input/GEO/Von_Hippel_Lindau/GSE33093"

# Output paths
out_data_file = "../../output/preprocess/Von_Hippel_Lindau/GSE33093.csv"
out_gene_data_file = "../../output/preprocess/Von_Hippel_Lindau/gene_data/GSE33093.csv"
out_clinical_data_file = "../../output/preprocess/Von_Hippel_Lindau/clinical_data/GSE33093.csv"
json_path = "../../output/preprocess/Von_Hippel_Lindau/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Meta-analysis of Clear Cell Renal Cell Carcinoma Gene Expression Defines a Variant Subgroup and Identifies Gender Influences on Tumor Biology"
!Series_summary	"Clear cell renal cell carcinoma comprises two dominant subtypes, ccA and ccB, with gender disparity providing additional disease information. A third minor subgroup has distinct expression profiles consistent with von Hippel-Lindau wild type status and displays variant histology features."
!Series_overall_design	"44 new tumor samples and six large, publicly available, ccRCC gene expression databases were identified that cumulatively provided data for 480 tumors for metaanalysis via meta-array compilation."
Sample Characteristics Dictionary:
{0: ['organism part: Kidney'], 1: ['histology: Clear Cell'], 2: ['biosource type: Frozen Sample'], 3: ['biosource provider: University of North Carolina']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
from typing import Optional, Dict, Any, Callable
import json

# 1. Gene Expression Data Availability
# Based on the Series_title and Series_summary, this dataset appears to be about gene expression 
# in Clear Cell Renal Cell Carcinoma, not just miRNA or methylation data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait (VHL status)
# Looking at the sample characteristics, it doesn't explicitly mention VHL status
# The Series_summary does mention "A third minor subgroup has distinct expression profiles consistent with von Hippel-Lindau wild type status"
# But there's no specific row in the sample characteristics that captures this information
trait_row = None  # Not available in the sample characteristics

# 2.2 Age data
# There's no age information provided in the sample characteristics
age_row = None

# 2.3 Gender data
# There's no gender information provided in the sample characteristics
gender_row = None

# Define conversion functions
def convert_trait(value):
    # Since trait_row is None, we don't need this function
    # But we'll define it for completeness
    if not value or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip().lower()
    if 'vhl' in value or 'von hippel-lindau' in value:
        if 'wild' in value or 'normal' in value:
            return 0  # Wild type (normal)
        elif 'mut' in value or 'abnormal' in value:
            return 1  # Mutant (disease)
    return None

def convert_age(value):
    # Since age_row is None, we don't need this function
    # But we'll define it for completeness
    if not value or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    # Since gender_row is None, we don't need this function
    # But we'll define it for completeness
    if not value or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip().lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


A new JSON file was created at: ../../output/preprocess/Von_Hippel_Lindau/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers from the previous step output
# The identifiers shown are: '1', '2', '3', '4'... - these are numeric identifiers
# These are not standard human gene symbols (which would be alphanumeric like "TP53", "BRCA1", etc.)
# These appear to be some type of numeric IDs (possibly probe IDs or array positions)
# Therefore, they will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1.0, 2.0, 3.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene annotation and determine the relevant columns

# Looking at the annotation preview, I can see:
# - 'ID' column appears to have the same kind of numeric identifiers ('1', '2', '3'...) 
#   as the gene expression data index
# - Several potential gene symbol columns: 'GENE', 'GENE_SYMBOL', 'NAME'
# - 'GENE_SYMBOL' seems the most appropriate for standard gene symbols

# 2. Create a mapping dataframe with 'ID' and 'GENE_SYMBOL' columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')

# Check how many mappings we have
print(f"Number of probe-to-gene mappings: {len(gene_mapping)}")

# Preview the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First few gene symbols: {list(gene_data.index[:10])}")


Number of probe-to-gene mappings: 32696
Gene mapping preview:
{'ID': ['12', '14', '15', '16', '18'], 'Gene': ['APOBEC3B', 'ATP11B', 'LOC100132006', 'DNAJA1', 'EHMT2']}
Gene expression data after mapping:
Shape: (18379, 44)
First few gene symbols: ['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Create a clinical dataframe with sample IDs from gene data
sample_ids = normalized_gene_data.columns.tolist()
print(f"Number of samples from gene data: {len(sample_ids)}")
print(f"First few sample IDs: {sample_ids[:5]}")

# From the background information, we know this dataset has ccRCC samples with VHL subtypes
# The information mentions "A third minor subgroup has distinct expression profiles consistent
# with von Hippel-Lindau wild type status"
# Since we can't identify which samples are which from the clinical data, we'll set a constant
# trait value for all samples (this will be filtered out as biased in quality check)
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df[trait] = 1  # Set all samples with the trait (this is likely biased)

print(f"Clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for {len(unbiased_linked_data)} ccRCC samples, but no VHL status information could be extracted from the sample metadata."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (17901, 44)
First few normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


Normalized gene data saved to ../../output/preprocess/Von_Hippel_Lindau/gene_data/GSE33093.csv
Number of samples from gene data: 44
First few sample IDs: ['GSM820734', 'GSM820735', 'GSM820736', 'GSM820737', 'GSM820738']
Clinical data shape: (44, 1)
           Von_Hippel_Lindau
GSM820734                  1
GSM820735                  1
GSM820736                  1
GSM820737                  1
GSM820738                  1
Clinical data saved to ../../output/preprocess/Von_Hippel_Lindau/clinical_data/GSE33093.csv
Linked data shape: (44, 17902)
           Von_Hippel_Lindau      A1BG      A1CF       A2M     A2ML1  \
GSM820734                1.0 -0.949890 -0.445831 -0.410555  0.284831   
GSM820735                1.0 -1.517597  0.393902  0.677915  0.188894   
GSM820736                1.0 -1.029776  0.053024  0.731052  0.538284   
GSM820737                1.0 -1.101823  0.144525  1.038360  0.288774   
GSM820738                1.0 -1.201518  0.685986  0.998316  0.026245   

            A3GALT2  

Shape after handling missing values: (44, 17902)
Quartiles for 'Von_Hippel_Lindau':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Von_Hippel_Lindau' in this dataset is severely biased.

Data quality check failed. The dataset is not suitable for association studies.
